# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 11** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [75]:
import findspark
findspark.init()

#### Spark Conexion


In [76]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Decision-Trees") \
    .master("spark://2dc725a6f972:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Data preparation

In [77]:
from team_name.spark_utils import SparkUtils

csv_path = "/home/jovyan/notebooks/data/IrisData/Iris.csv"

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([
    ("Id", "integer"),
    ("SepalLengthCm", "float"),
    ("SepalWidthCm","float"),
    ("PetalLengthCm","float"),
    ("PetalWidthCm","float"),
    ("Species","string")
])


# Convert list to a DataFrame
df = spark.read \
        .schema(schema) \
        .option("header", "true") \
        .csv(csv_path)

df = df.dropna(how="any").drop("Id")
df.show(10)

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
+-------------+------------+-------------+------------+-----------+
only showing top 10 rows



#### String indexer of Species

In [78]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol="Species", outputCol="label")
df = label_indexer.fit(df).transform(df).drop("Species")
df.show(10)

+-------------+------------+-------------+------------+-----+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|label|
+-------------+------------+-------------+------------+-----+
|          5.1|         3.5|          1.4|         0.2|  0.0|
|          4.9|         3.0|          1.4|         0.2|  0.0|
|          4.7|         3.2|          1.3|         0.2|  0.0|
|          4.6|         3.1|          1.5|         0.2|  0.0|
|          5.0|         3.6|          1.4|         0.2|  0.0|
|          5.4|         3.9|          1.7|         0.4|  0.0|
|          4.6|         3.4|          1.4|         0.3|  0.0|
|          5.0|         3.4|          1.5|         0.2|  0.0|
|          4.4|         2.9|          1.4|         0.2|  0.0|
|          4.9|         3.1|          1.5|         0.1|  0.0|
+-------------+------------+-------------+------------+-----+
only showing top 10 rows



### Assemble the features into a single vector column

In [79]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(df).select("label", "features")

### Split the data into training and test sets 80% training data and 20% testing data

In [80]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [81]:
print("Original Dataset")
data_with_features.show(6)

# Print train dataset
print("train set")
train_df.show(6)

Original Dataset
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[5.09999990463256...|
|  0.0|[4.90000009536743...|
|  0.0|[4.69999980926513...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.5999999046...|
|  0.0|[5.40000009536743...|
+-----+--------------------+
only showing top 6 rows

train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[4.30000019073486...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.5,2.2999999523...|
|  0.0|[4.59999990463256...|
+-----+--------------------+
only showing top 6 rows



---
# Decision Tree Model

#### Create a Decision Tree model and OnevsAll

In [82]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import OneVsRest

# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
ovr_tree = OneVsRest(classifier=dt)


### Trainning

In [83]:
# Training, jeje
dt_model = ovr_tree.fit(train_df)

#### Predictions

In [84]:
# Use the trained model to make predictions on the test data
predictions_dt = dt_model.transform(test_df)

# Show predictions
predictions_dt.select("features", "prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,3.4000000953...|       0.0|
|[5.0,3.5,1.299999...|       0.0|
|[5.09999990463256...|       0.0|
|[5.19999980926513...|       0.0|
|[5.40000009536743...|       0.0|
|[5.69999980926513...|       1.0|
|[5.80000019073486...|       1.0|
|[5.80000019073486...|       1.0|
|[6.09999990463256...|       1.0|
|[6.09999990463256...|       1.0|
|[6.19999980926513...|       1.0|
|[6.30000019073486...|       1.0|
|[6.69999980926513...|       1.0|
|[4.90000009536743...|       1.0|
|[6.0,3.0,4.800000...|       2.0|
|[6.09999990463256...|       2.0|
|[6.19999980926513...|       2.0|
|[6.30000019073486...|       2.0|
|[6.30000019073486...|       2.0|
|[6.40000009536743...|       2.0|
+--------------------+----------+
only showing top 20 rows



---
# Support Vector Machines 

#### Create Support Vector Machines and OnevsAll

In [85]:
from pyspark.ml.classification import LinearSVC

# Initialize and train the Decision SVM
lsvc = LinearSVC(maxIter=10, regParam=0.01)
ovr_vector = OneVsRest(classifier=lsvc)

#### Training

In [86]:
svm_model = ovr_vector.fit(data_with_features)  

#### Predictions

In [87]:
predictions_svm = svm_model.transform(test_df)
predictions_dt.select("features", "prediction").show(20)

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,3.4000000953...|       0.0|
|[5.0,3.5,1.299999...|       0.0|
|[5.09999990463256...|       0.0|
|[5.19999980926513...|       0.0|
|[5.40000009536743...|       0.0|
|[5.69999980926513...|       1.0|
|[5.80000019073486...|       1.0|
|[5.80000019073486...|       1.0|
|[6.09999990463256...|       1.0|
|[6.09999990463256...|       1.0|
|[6.19999980926513...|       1.0|
|[6.30000019073486...|       1.0|
|[6.69999980926513...|       1.0|
|[4.90000009536743...|       1.0|
|[6.0,3.0,4.800000...|       2.0|
|[6.09999990463256...|       2.0|
|[6.19999980926513...|       2.0|
|[6.30000019073486...|       2.0|
|[6.30000019073486...|       2.0|
|[6.40000009536743...|       2.0|
+--------------------+----------+
only showing top 20 rows



---
# Models Testing

In [88]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

### Tree
print("\n *** TREE ***")
accuracy = evaluator.evaluate(predictions_dt, {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_dt, {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")  

### Vector
print("\n *** VECTOR ***")
accuracy = evaluator.evaluate(predictions_svm, {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_svm, {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_svm, {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_svm, {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}") 


 *** TREE ***


Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545

 *** VECTOR ***
Accuracy: 0.8636363636363636
Precision: 0.8662337662337662
Recall: 0.8636363636363636
F1 Score: 0.8626794258373206


In [ ]:
sc.stop()